In [11]:
import os
from PIL import Image
import numpy as np
from google.colab import drive
import random
import math

# 구글 드라이브 마운트
drive.mount('/content/drive')

# 클래스별 폴더 경로
class_folders = ['BLDC-400-Back', 'BLDC-400-Front', 'G2', 'CN7']
base_path = '/content/drive/MyDrive/data'

def rotate_image(image, angle):
    return image.rotate(angle, expand=False)

def print_stats(split_name, images_per_class, aug_methods_count):
    print(f"\n=== {split_name} 데이터 통계 ===")
    total_images = 0
    for class_name in class_folders:
        print(f"\n{class_name} 클래스:")
        print(f"총 이미지 수: {images_per_class}")
        print("증강법별 이미지 수:")
        for method, count in aug_methods_count[class_name].items():
            print(f"- {method}: {count}장")
        total_images += images_per_class
    print(f"\n총 {split_name} 이미지 수: {total_images}장")

def augment_data(input_path, output_path, split_type):
    # 목표 이미지 수 설정
    if split_type == 'train':
        target_per_class = 245
        rotation_range = 30
        fixed_angles = [90, 180, 270]
    elif split_type == 'val':
        target_per_class = 35
        rotation_range = 30
        fixed_angles = [90, 180, 270]
    else:  # test
        target_per_class = 70
        rotation_range = 45
        fixed_angles = [45, 90, 135, 180, 225, 270, 315]

    aug_methods_count = {class_name: {
        '원본': 0,
        f'±{rotation_range}도 랜덤': 0,
        '고정각도': 0
    } for class_name in class_folders}

    # 각 클래스별 처리
    for class_name in class_folders:
        class_path = os.path.join(input_path, class_name)
        output_class_path = os.path.join(output_path, split_type, class_name)
        os.makedirs(output_class_path, exist_ok=True)

        # 원본 이미지 목록
        images = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

        # 필요한 증강 횟수 계산
        total_needed = target_per_class
        images_per_method = math.ceil(total_needed / 3)

        # 1. 원본 이미지 저장
        img_count = 0
        while img_count < images_per_method:
            for img_name in images:
                if img_count >= images_per_method:
                    break
                img = Image.open(os.path.join(class_path, img_name))
                save_path = os.path.join(output_class_path, f'original_{img_count}_{img_name}')
                img.save(save_path)
                img_count += 1
                aug_methods_count[class_name]['원본'] += 1

        # 2. 랜덤 회전
        img_count = 0
        while img_count < images_per_method:
            for img_name in images:
                if img_count >= images_per_method:
                    break
                img = Image.open(os.path.join(class_path, img_name))
                angle = random.uniform(-rotation_range, rotation_range)
                rotated = rotate_image(img, angle)
                save_path = os.path.join(output_class_path, f'random_rot_{img_count}_{img_name}')
                rotated.save(save_path)
                img_count += 1
                aug_methods_count[class_name][f'±{rotation_range}도 랜덤'] += 1

        # 3. 고정 각도 회전
        img_count = 0
        angle_index = 0
        while img_count < total_needed - (2 * images_per_method):
            for img_name in images:
                if img_count >= (total_needed - (2 * images_per_method)):
                    break
                img = Image.open(os.path.join(class_path, img_name))
                angle = fixed_angles[angle_index % len(fixed_angles)]
                rotated = rotate_image(img, angle)
                save_path = os.path.join(output_class_path, f'fixed_rot_{angle}_{img_count}_{img_name}')
                rotated.save(save_path)
                img_count += 1
                aug_methods_count[class_name]['고정각도'] += 1
                angle_index += 1

    return aug_methods_count

# 메인 실행
if __name__ == "__main__":
    # 출력 디렉토리 생성
    output_base = '/content/drive/MyDrive/augmented_data'
    os.makedirs(output_base, exist_ok=True)

    # train, val, test 각각 증강
    splits = ['train', 'val', 'test']
    target_counts = {'train': 245, 'val': 35, 'test': 70}

    for split in splits:
        print(f"\n{split} 데이터 증강 시작...")
        aug_methods_count = augment_data(base_path, output_base, split)
        print_stats(split, target_counts[split], aug_methods_count)

    print("\n증강 완료!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

train 데이터 증강 시작...

=== train 데이터 통계 ===

BLDC-400-Back 클래스:
총 이미지 수: 245
증강법별 이미지 수:
- 원본: 82장
- ±30도 랜덤: 82장
- 고정각도: 81장

BLDC-400-Front 클래스:
총 이미지 수: 245
증강법별 이미지 수:
- 원본: 82장
- ±30도 랜덤: 82장
- 고정각도: 81장

G2 클래스:
총 이미지 수: 245
증강법별 이미지 수:
- 원본: 82장
- ±30도 랜덤: 82장
- 고정각도: 81장

CN7 클래스:
총 이미지 수: 245
증강법별 이미지 수:
- 원본: 82장
- ±30도 랜덤: 82장
- 고정각도: 81장

총 train 이미지 수: 980장

val 데이터 증강 시작...

=== val 데이터 통계 ===

BLDC-400-Back 클래스:
총 이미지 수: 35
증강법별 이미지 수:
- 원본: 12장
- ±30도 랜덤: 12장
- 고정각도: 11장

BLDC-400-Front 클래스:
총 이미지 수: 35
증강법별 이미지 수:
- 원본: 12장
- ±30도 랜덤: 12장
- 고정각도: 11장

G2 클래스:
총 이미지 수: 35
증강법별 이미지 수:
- 원본: 12장
- ±30도 랜덤: 12장
- 고정각도: 11장

CN7 클래스:
총 이미지 수: 35
증강법별 이미지 수:
- 원본: 12장
- ±30도 랜덤: 12장
- 고정각도: 11장

총 val 이미지 수: 140장

test 데이터 증강 시작...

=== test 데이터 통계 ===

BLDC-400-Back 클래스:
총 이미지 수: 70
증강법별 이미지 수:
- 원본: 24장
- ±45도 랜덤: 24장
- 고정각도: 22장

BLDC-400-Front 클래